<a href="https://colab.research.google.com/github/bdoma97/diplomamunka/blob/main/generate_characters_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/_75_all /mydrive
!ls /mydrive

Mounted at /content/gdrive
fine_tuning_section_2	      predicition_M_eredeti.csv
legyszives_1024_5	      predicition_M_fgsm_0.001.csv
legyszives_1024_7	      predicition_M_fgsm_0.01.csv
legyszives_128_5	      predicition_M_fgsm_0.1.csv
legyszives_128_7	      predicition_N_bim_0.001.csv
legyszives_2048_5	      predicition_N_bim_0.01.csv
legyszives_2048_7	      predicition_N_bim_0.1.csv
legyszives_256_5	      predicition_N_cw2.csv
legyszives_256_7	      predicition_N_eredeti.csv
legyszives_512_5	      predicition_N_fgsm_0.001.csv
legyszives_512_7	      predicition_N_fgsm_0.01.csv
predicition_A_bim_0.001.csv   predicition_N_fgsm_0.1.csv
predicition_A_bim_0.01.csv    predicition_O_bim_0.001.csv
predicition_A_bim_0.1.csv     predicition_O_bim_0.01.csv
predicition_A_cw2.csv	      predicition_O_bim_0.1.csv
predicition_A_eredeti.csv     predicition_O_cw2.csv
predicition_A_fgsm_0.001.csv  predicition_O_eredeti.csv
predicition_A_fgsm_0.01.csv   predicition_O_fgsm_0.001.csv
predicition_A_fgsm_0.1

In [ ]:
#!pip install -q tensorflow==2.0.0b1
!pip install -q tensorflow==2.2
import tensorflow as tf

     |████████████████████████████████| 516.2MB 35kB/s 
     |████████████████████████████████| 460kB 53.7MB/s 
     |████████████████████████████████| 3.0MB 50.0MB/s 


In [ ]:
print("\nTensorflow Version: " + tf.__version__)
!nvidia-smi -L


Tensorflow Version: 2.2.0
GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-e83d6e48-e1b5-3257-70ce-646f1563054e)


In [ ]:
import keras
from tensorflow.keras.models import load_model

model_ = keras.models.load_model("gdrive/MyDrive/_75_all/fine_tuning_section_2/last_epoch_063_model.h5")
model_.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 37, 37, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 37, 37, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

import cleverhans

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-zhfajz1w/cleverhans
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-zhfajz1w/cleverhans
     |████████████████████████████████| 163kB 14.2MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for cleverhans: filename=cleverhans-4.0.0-cp37-none-any.whl size=92450 sha256=699ac28d8a174f77bee5f9b9582b019af6d6517a906719e7b508f589a7f3138c
  Stored in directory: /tmp/pip-ephem-wheel-cache-bgmwx62a/wheels/6e/59/ec/723a6f654aaf62c8c40f0f0850fdf71a4948598697f56c3bfa
Successfully built cleverhans


In [ ]:
logits_model = tf.keras.Model(model_.input,model_.layers[-1].output)
logits_model

In [ ]:
import glob
import random

images = glob.glob("gdrive/MyDrive/_75_all/validation/*/*")
random.seed(42)
random.shuffle(images)
print(len(images))

2700


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import random
from cleverhans.tf2.attacks import fast_gradient_method

images = glob.glob("gdrive/MyDrive/_75_all/validation/*/*")
random.seed(42)
random.shuffle(images)
print(len(images))

print('FGSM - 0.001')
counter = 0
target_time = 0
for i in images[:450]:
  if counter %50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, 0.001, np.inf, targeted=False)
  elapsed_time = time.process_time() - t
  target_time += elapsed_time

  image_ = np.squeeze(image)
  image_ = array_to_img(image_)
  image_2 = np.squeeze(adv_example_untargeted_label)
  image_2 = array_to_img(image_2)
  
  image_folder = i.split('/')[4]
  image_name = i.split('/')[5]
  image_name = image_name.split('.')[0]
  saving_name = 'gdrive/MyDrive/_75_all/validation/_' + image_folder + '/FGSM_0_001_' + image_name + '.png'
  image_2.save(saving_name)
  counter += 1
print('FGSM - 0.001 támadás {}'.format(target_time))


print('FGSM - 0.1')
counter = 0
target_time = 0
for i in images[450:900]:
  if counter %50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, 0.1, np.inf, targeted=False)
  elapsed_time = time.process_time() - t
  target_time += elapsed_time

  image_ = np.squeeze(image)
  image_ = array_to_img(image_)
  image_2 = np.squeeze(adv_example_untargeted_label)
  image_2 = array_to_img(image_2)
  
  image_folder = i.split('/')[4]
  image_name = i.split('/')[5]
  image_name = image_name.split('.')[0]
  saving_name = 'gdrive/MyDrive/_75_all/validation/_' + image_folder + '/FGSM_0_1_' + image_name + '.png'
  image_2.save(saving_name)
  counter += 1
print('FGSM - 0.1 támadás {}'.format(target_time))

from cleverhans.tf2.attacks import projected_gradient_descent

print('BIM - 0.001')
target_time = 0
counter = 0
for i in images[900:1350]:
  if counter %50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=0.001, eps_iter=0.001,
                                                                                       nb_iter=3, norm=np.inf, targeted=False)
  elapsed_time = time.process_time() - t
  target_time += elapsed_time

  image_ = np.squeeze(image)
  image_ = array_to_img(image_)
  image_2 = np.squeeze(adv_example_untargeted_label)
  image_2 = array_to_img(image_2)
  
  image_folder = i.split('/')[4]
  image_name = i.split('/')[5]
  image_name = image_name.split('.')[0]
  saving_name = 'gdrive/MyDrive/_75_all/validation/_' + image_folder + '/BIM_0_001_' + image_name + '.png'
  image_2.save(saving_name)
  counter += 1
print('BIM - 0.001 támadás {}'.format(target_time))

print('BIM - 0.1')
target_time = 0
counter = 0
for i in images[1350:1800]:
  if counter %50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=0.1, eps_iter=0.1,
                                                                                       nb_iter=3, norm=np.inf, targeted=False)
  elapsed_time = time.process_time() - t
  target_time += elapsed_time

  image_ = np.squeeze(image)
  image_ = array_to_img(image_)
  image_2 = np.squeeze(adv_example_untargeted_label)
  image_2 = array_to_img(image_2)
  
  image_folder = i.split('/')[4]
  image_name = i.split('/')[5]
  image_name = image_name.split('.')[0]
  saving_name = 'gdrive/MyDrive/_75_all/validation/_' + image_folder + '/BIM_0_1_' + image_name + '.png'
  image_2.save(saving_name)
  counter += 1
print('BIM - 0.1 támadás {}'.format(target_time))


from cleverhans.tf2.attacks import carlini_wagner_l2

print('CW2')
target_time = 0
counter = 0
for i in images[1800:]:
  if counter %50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  target_time += elapsed_time

  image_ = np.squeeze(image)
  image_ = array_to_img(image_)
  image_2 = np.squeeze(adv_example_untargeted_label)
  image_2 = array_to_img(image_2)
  
  image_folder = i.split('/')[4]
  image_name = i.split('/')[5]
  image_name = image_name.split('.')[0]
  saving_name = 'gdrive/MyDrive/_75_all/validation/_' + image_folder + '/CW_' + image_name + '.png'
  image_2.save(saving_name)
  counter += 1
print('CW2 {}'.format(target_time))

2700
FGSM - 0.001
0 50 100 150 200 250 300 350 400 FGSM - 0.001 támadás 139.12987302603506
FGSM - 0.1
0 50 100 150 200 250 300 350 400 FGSM - 0.1 támadás 140.0861016019917
BIM - 0.001
0 50 100 150 200 250 300 350 400 BIM - 0.001 támadás 167.30540257898974
BIM - 0.1
0 50 100 150 200 250 300 350 400 BIM - 0.1 támadás 167.25840677504675
CW2
0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 CW2 7579.031306223977
